In [21]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_community.chat_models import ChatOllama

In [22]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], template=template
    )
    llm = ChatOllama(model='qwen2', temperature=0)
    memory = ConversationBufferWindowMemory()

    chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

    return chain

def initialize_meta_chain():
    meta_template = """
    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as possible. Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions. Don't forget any important details in the current Instructions! Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], template=meta_template
    )

    llm = ChatOllama(model='qwen2', temperature=0)
    meta_chain = LLMChain(llm=llm, prompt=meta_prompt)
    return meta_chain

def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history

def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter) :]
    return new_instructions

def main(task, max_iters=3, max_meta_iters=5):
    failed_phrase = "task failed"
    success_phrase = "task succeeded"
    key_phrases = [success_phrase, failed_phrase]

    instructions = "None"
    for i in range(max_meta_iters):
        print(f"[Episode {i+1}/{max_meta_iters}]")
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f"(Step {j+1}/{max_iters})")
            print(f"Assistant: {output}")
            print("Human: ")
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print("You succeeded! Thanks for playing!")
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f"Feedback: {meta_output}")
        instructions = get_new_instructions(meta_output)
        print(f"New Instructions: {instructions}")
        print("\n" + "#" * 80 + "\n")
    print("You failed! Thanks for playing!")


In [23]:
task = "Provide a systematic argument for why we should always eat pasta with olives."
main(task)

[Episode 1/5]
(Step 1/3)
Assistant: Eating pasta with olives can be considered beneficial and enjoyable based on several systematic arguments:

1. **Nutritional Value**: Olives are rich in monounsaturated fats, which are known to have heart-healthy benefits. Consuming foods high in these fats may help reduce the risk of cardiovascular disease by lowering cholesterol levels and improving blood pressure. When olives are added to pasta dishes, they contribute a healthy fat source that can enhance the overall nutritional profile of your meal.

2. **Flavor Enhancement**: Olives add a unique flavor dimension to pasta dishes. Their briny taste complements many sauces and ingredients commonly used in pasta recipes. The combination of salty olives with sweet tomatoes or creamy cheeses creates a balanced palate experience, making meals more satisfying and enjoyable.

3. **Versatility**: Pasta dishes can be adapted to various dietary preferences and restrictions by incorporating different types o

 Give it single lined


(Step 2/3)
Assistant: Eating pasta with olives offers numerous benefits including enhanced nutrition, flavor, cultural significance, sustainability, health promotion, satisfaction, and culinary pleasure.
Human: 


 what else can you tell about it?


(Step 3/3)
Assistant: Eating pasta with olives also supports heart health due to monounsaturated fats in olives, adds variety to your diet by introducing new textures and flavors, and can be a part of a balanced meal plan for those following specific dietary guidelines.
Human: 


...

Feedback: Critique: The Assistant provided a detailed argument for why we should eat pasta with olives, which was thorough and informative. However, it could have been more concise and focused on answering the user's question directly without providing additional information about cooking or eating in general. The Assistant might want to remember that this user prefers quick responses and direct answers.

Instructions: When responding to questions about food combinations or nutrition, focus on providing a clear, concise answer based on the specific query asked by the user. Avoid adding unnecessary details or context unless directly requested. If you need to provide additional information, ensure it is relevant and directly addresses the user's question.
New Instructions: When responding to questions about food combinations or nutrition, focus on providing a clear, concise answer based on the specific query asked by the user. Avoid adding unnecessary details or context unless directly r

 task succeed


(Step 2/3)
Assistant: Yes, the task was successful. I provided a systematic argument for why eating pasta with olives is beneficial based on nutritional advantages such as heart health, antioxidant content, vitamin E intake, calcium absorption enhancement, digestive health promotion, anti-inflammatory properties, and alignment with the Mediterranean diet benefits. The response was clear, concise, and directly addressed the user's query without adding unnecessary details or context.
Human: 


 task succeeded


You succeeded! Thanks for playing!


In [31]:
from langchain_core.runnables import RunnableSequence

def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], template=template
    )
    llm = ChatOllama(model='qwen2', temperature=0)

    chain = RunnableSequence(prompt, llm)

    return chain, memory

def initialize_meta_chain():
    meta_template = """
    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as possible. Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions. Don't forget any important details in the current Instructions! Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], template=meta_template
    )

    llm = ChatOllama(model='qwen2', temperature=0)
    meta_chain = RunnableSequence(meta_prompt, llm)
    return meta_chain

def get_chat_history(memory):
    memory_key = memory.memory_key
    chat_history = memory.load_memory_variables(memory_key)[memory_key]
    return chat_history

def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter) :]
    return new_instructions

def main(task, max_iters=3, max_meta_iters=5):
    failed_phrase = "task failed"
    success_phrase = "task succeeded"
    key_phrases = [success_phrase, failed_phrase]

    instructions = "None"
    for i in range(max_meta_iters):
        print(f"[Episode {i+1}/{max_meta_iters}]")
        chain, memory = initialize_chain(instructions, memory=None)
        output = chain.invoke({"history": "", "human_input": task})
        print(f"Assistant: {output.content}")
        memory.save_context({"human_input": task}, {"content": output.content})
        for j in range(max_iters):
            print(f"(Step {j+1}/{max_iters})")
            print("Human: ")
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.invoke({"history": get_chat_history(memory), "human_input": human_input})
            print(f"Assistant: {output.content}")
            memory.save_context({"human_input": human_input}, {"content": output.content})
        if success_phrase in human_input.lower():
            print("You succeeded! Thanks for playing!")
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.invoke({"chat_history": get_chat_history(memory)})
        print(f"Feedback: {meta_output.content}")
        instructions = get_new_instructions(meta_output.content)
        print(f"New Instructions: {instructions}")
        print("\n" + "#" * 80 + "\n")
    print("You failed! Thanks for playing!")

# Example usage
task = "Provide a systematic argument for why we should always eat pasta with olives."
main(task)


[Episode 1/5]
Assistant: Eating pasta with olives can be considered beneficial and enjoyable based on several systematic arguments:

1. **Nutritional Value**: Olives are rich in monounsaturated fats, which are known to have heart-healthy benefits. Consuming foods high in these fats may help reduce the risk of cardiovascular disease by lowering cholesterol levels and improving blood pressure. When olives are added to pasta dishes, they contribute a healthy fat source that can enhance the overall nutritional profile of your meal.

2. **Flavor Enhancement**: Olives add a unique flavor dimension to pasta dishes. Their briny taste complements many sauces and ingredients commonly used in pasta recipes. The combination of salty olives with sweet tomatoes or creamy cheeses creates a balanced palate experience, making meals more satisfying and enjoyable.

3. **Versatility**: Pasta dishes can be adapted to various dietary preferences and restrictions by incorporating different types of olives (g

 give summary


Assistant: Eating pasta with olives offers several benefits:

1. **Nutritional Value**: Olives provide heart-healthy monounsaturated fats, enhancing the nutritional profile of your meal.

2. **Flavor Enhancement**: They add a unique salty flavor that complements various sauces and ingredients in pasta dishes, making meals more enjoyable.

3. **Versatility**: The ability to customize with different types of olives caters to diverse dietary preferences and restrictions.

4. **Cultural Significance**: Incorporating olives into pasta dishes connects individuals to culinary traditions from around the world.

5. **Sustainability**: Choosing locally sourced olives supports local economies and promotes environmental responsibility.

6. **Health Benefits**: Olive oil, often used in such recipes, is rich in antioxidants that contribute to overall health when consumed regularly.

7. **Satisfaction and Pleasure**: The combination of flavors and textures provides immediate satisfaction and contribu

 good


Assistant: Thank you! If you have any other questions or need further information on this topic or anything else, feel free to ask. I'm here to help!
(Step 3/3)
Human: 


 task succeeded


You succeeded! Thanks for playing!
